In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys, os
from datetime import date
from dotenv import load_dotenv

load_dotenv("../api.env")
sys.path.append(os.path.abspath("../src"))


import importlib
import temp_visualisering
importlib.reload(temp_visualisering)



from klimagass_visualisering import *
from temp_visualisering import *

import importlib
import temp_visualisering
importlib.reload(temp_visualisering)
from temp_visualisering import *

from statistikk import analyser_fil

# Henter inn data: 
statistikk_norge, df_total_norge, df_norge_alt = analyser_fil("../data/klimagassutslipp_norge_renset.csv", datokolonne="år", groupby="år")
statistikk_verden, df_total_verden, df_verden_alt = analyser_fil( "../data/klimagassutslipp_verden_renset.csv", datokolonne="År", groupby="år")

årlig_temp_df, tiårs_temp_df = load_and_compute("../data/temp_gloshaugen_historisk_renset_ 50.csv")
sti="../data/temp_gloshaugen_historisk_renset_ 50.csv"


# Rens kolonner
df_norge = df_total_norge[df_total_norge["kilde_(aktivitet)"].str.lower().str.contains("alle kilder")].copy()
df_norge = df_norge[["år", "utslipp_til_luft_(1_000_tonn_co2-ekvivalenter,_ar5)"]]
df_norge.rename(columns={"utslipp_til_luft_(1_000_tonn_co2-ekvivalenter,_ar5)": "gjennomsnitt"}, inplace=True)
df_norge_alt = df_norge_alt.rename(columns={"utslipp_til_luft_(1_000_tonn_co2-ekvivalenter,_ar5)": "gjennomsnitt"})
df_verden = df_verden_alt.rename(columns={"utslipp_i_co2_ekvivalenter": "gjennomsnitt"})


# Ulike widgets
valg = widgets.ToggleButtons(
    options=["Temperaturdata", "Klimagassutslipp", "Luftkvalitet"],
    description='Analyser:',
    button_style='info'
)

sted_widget = widgets.Dropdown(
    options=['Trykk her for å velge', 'Norge', 'Globalt', 'Sammenlign'],
    value='Trykk her for å velge',
    description='Sted:'
)

temperaturtype_widget = widgets.Dropdown(
    options=["Trykk her for å velge", "Historiske data", "Sanntidsdata"],
    value="Trykk her for å velge",
    description="Datakilde:"
)

år_widget = widgets.IntSlider(
    description="Velg år:",
    min=df_norge_alt["år"].min(),
    max=df_norge_alt["år"].max(),
    value=df_norge_alt["år"].min(), 
    continuous_update=False
)

temp_plott_valg = widgets.Dropdown(
    options=["Trykk her for å velge",
             "Årlig utvikling", 
             "Gjennomsnitt per tiår",
            "Avvik fra snitt", 
            "Endring total (SOL)"],
    value="Trykk her for å velge",
    description="Temperatur:"
)
sanntid_plott_valg = widgets.Dropdown(
    options=["Trykk her for å velge", "Temperatur neste dager"], 
    value="Trykk her for å velge",
    description="Plott:"
)



plott_valg = widgets.Dropdown(description="Velg plott:")
ny_graf_knapp = widgets.Button(description="Se ny graf", button_style='warning')
output = widgets.Output()

# Funksjoner

def vis_kake_plot(e):
    if e['name'] == 'value':
        output.clear_output(wait=True)
        with output:
            display(år_widget)
            plot_andel_per_kilde(df_norge_alt, år_widget.value)
            display(ny_graf_knapp)

def oppdater_plottvalg(endring=None):
    if sted_widget.value == "Norge":
        plott_valg.options = [
            "Utslipp per år",
            "Utslipp per tiår",
            "Utslipp per kilde (linjeplot)",
            "Utslipp per kilde (kakediagram for et år)",
            "Heatmap per kilde"
        ]
    elif sted_widget.value == "Globalt":
        plott_valg.options = ["Utslipp per år", 
                              "Utslipp per tiår"]
    elif sted_widget.value == "Sammenlign":
        plott_valg.options = ["Samme graf", 
                              "Separate grafer"]

def vis_valgt_data(endring):
    output.clear_output()
    if valg.value == "Klimagassutslipp":
        with output:
            display(sted_widget)
            oppdater_plottvalg()
            display(plott_valg)
    elif valg.value == "Temperaturdata":
        with output:    
            display(temperaturtype_widget)
        
            if temperaturtype_widget.value == "Historiske data":
                display(temp_plott_valg)
            elif temperaturtype_widget.value == "Sanntidsdata":
                display(sanntid_plott_valg)
            

def vis_klimagass_plot(endring):
    if endring['name'] == 'value':
        output.clear_output(wait=True)
        with output:
            if plott_valg.value == "Utslipp per år":
                df = df_norge if sted_widget.value == "Norge" else df_verden
                plot_utslipp(df, tittel=f"Årlige utslipp i {sted_widget.value}")

            elif plott_valg.value == "Utslipp per tiår":
                df = df_norge if sted_widget.value == "Norge" else df_verden
                plot_utslipp_per_tiar(df)

            elif plott_valg.value == "Utslipp per kilde (linjeplot)":
                plot_utslipp_per_kilde(df_norge_alt)

            elif plott_valg.value == "Utslipp per kilde (kakediagram for et år)":
                år_widget.value = år_widget.min
                display(år_widget)
                år_widget.observe(vis_kake_plot, names='value')

                plot_andel_per_kilde(df_norge_alt, år_widget.value)
                display(ny_graf_knapp)
                
            elif plott_valg.value == "Heatmap per kilde":
                plot_heatmap_per_kilde(df_norge_alt)

            elif plott_valg.value == "Samme graf":
                plot_norge_og_verden_sammen(df_norge, df_verden)

            elif plott_valg.value == "Separate grafer":
                plot_norge_og_verden_separat(df_norge, df_verden)

            display(ny_graf_knapp)

def vis_temperatur_plot(endring):
    output.clear_output(wait=True)

    with output:
        if temperaturtype_widget.value == "Historiske data":
            valgt_plott = temp_plott_valg.value
            if valgt_plott == "Årlig utvikling":
                plot_temperatur_år(årlig_temp_df)
            elif valgt_plott == "Gjennomsnitt per tiår":
                tiår_df = tiår_snitt(årlig_temp_df)
                plot_by_decade(tiår_df)
            elif valgt_plott == "Avvik fra snitt":
                plot_avvik(årlig_temp_df)
            elif valgt_plott == "Endring total (SOL)" and sti:
                if not os.path.exists(sti):
                    print("Fant ikke temperaturfil.")
                    return
                tegn_endring_sol(sti)

        elif temperaturtype_widget.value == "Sanntidsdata":
            plot_sanntids_temperatur()

        display(ny_graf_knapp)



def tilbakestill_knapp_callback(_):
    output.clear_output()
    vis_valgt_data(None)



valg.observe(vis_valgt_data, names='value')
sted_widget.observe(oppdater_plottvalg, names='value')
temp_plott_valg.observe(vis_temperatur_plot, names='value')
sanntid_plott_valg.observe(vis_temperatur_plot, names='value')
temperaturtype_widget.observe(vis_valgt_data, names='value')
plott_valg.observe(vis_klimagass_plot, names='value')
ny_graf_knapp.on_click(tilbakestill_knapp_callback)



display(valg)
display(output)
vis_valgt_data(None)